Kaggle: https://www.kaggle.com/code/eisbarinkaltenpolar/homework-5
Wandb: https://wandb.ai/mlteamaau/pretraining-resnet18?workspace=user-qfaithencz

In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/cifar-10/trainLabels.csv
/kaggle/input/cifar-10/sampleSubmission.csv
/kaggle/input/cifar-10/test.7z
/kaggle/input/cifar-10/train.7z


## Import everything needed

In [2]:
import glob
from PIL import Image
import matplotlib.pyplot as plt
import collections
import math
import os
import shutil
from sklearn.model_selection import train_test_split
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import DataLoader, Dataset
import torchvision
from torchvision import datasets, transforms
np.random.seed(0)
torch.manual_seed(0)
torch.cuda.manual_seed(0)

/opt/conda/lib/python3.10/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.24.3
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


## Unzip datasets

In [3]:
!pip install py7zr

  Obtaining dependency information for py7zr from https://files.pythonhosted.org/packages/c0/39/71263be4c6081e1788f4da35e937fca1808914b0d98a04bd44255b948699/py7zr-0.20.8-py3-none-any.whl.metadata
  Obtaining dependency information for pycryptodomex>=3.16.0 from https://files.pythonhosted.org/packages/20/7a/3162173af8597f0399b45c6aaa4939ccae908476fdf1b3a3cc30631fc9fb/pycryptodomex-3.20.0-cp35-abi3-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata
  Obtaining dependency information for pyzstd>=0.15.9 from https://files.pythonhosted.org/packages/02/97/57de14ccb0d033464aabc5110eb2f9dc717c97e17f2ffc1235680a27e5de/pyzstd-0.15.9-cp310-cp310-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata
  Obtaining dependency information for pyppmd<1.2.0,>=1.1.0 from https://files.pythonhosted.org/packages/09/76/61db4268a439cfba8736b14130d928d199633fab2360a2c5043332a427d2/pyppmd-1.1.0-cp310-cp310-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata
  Obtaining dependency information for pybc

# WARNING: It can take a lot of time to uncompress!

In [4]:
!python -m py7zr x /kaggle/input/cifar-10/train.7z

In [5]:
!python -m py7zr x /kaggle/input/cifar-10/test.7z

In [6]:
data_dir = '/kaggle/working/'

In [7]:
def read_csv_labels(fname):
    """Read `fname` to return a filename to label dictionary."""
    with open(fname, 'r') as f:
        # Skip the file header line (column name)
        lines = f.readlines()[1:]
    tokens = [l.rstrip().split(',') for l in lines]
    return dict(((name, label) for name, label in tokens))

labels = read_csv_labels(os.path.join(data_dir, '/kaggle/input/cifar-10/trainLabels.csv'))
print(f'Number training examples: {len(labels)}')
print(f'Number classes: {len(set(labels.values()))}')

Number training examples: 50000
Number classes: 10


In [8]:
def copyfile(filename, target_dir):
    """Copy a file into a target directory."""
    os.makedirs(target_dir, exist_ok=True)
    shutil.copy(filename, target_dir)

def reorg_train_valid(data_dir, labels, valid_ratio):
    """Split the validation set out of the original training set."""
    # The number of examples of the class that has the fewest examples in the
    # training dataset
    n = collections.Counter(labels.values()).most_common()[-1][1]
    # The number of examples per class for the validation set
    n_valid_per_label = max(1, math.floor(n * valid_ratio))
    label_count = {}
    for train_file in os.listdir(os.path.join(data_dir, 'train')):
        label = labels[train_file.split('.')[0]]
        fname = os.path.join(data_dir, 'train', train_file)
        copyfile(fname, os.path.join(data_dir, 'train_valid_test',
                                     'train_valid', label))
        if label not in label_count or label_count[label] < n_valid_per_label:
            copyfile(fname, os.path.join(data_dir, 'train_valid_test',
                                         'valid', label))
            label_count[label] = label_count.get(label, 0) + 1
        else:
            copyfile(fname, os.path.join(data_dir, 'train_valid_test',
                                         'train', label))
    return n_valid_per_label

def reorg_test(data_dir):
    """Organize the testing set for data loading during prediction."""
    for test_file in os.listdir(os.path.join(data_dir, 'test')):
        copyfile(os.path.join(data_dir, 'test', test_file),
                 os.path.join(data_dir, 'train_valid_test', 'test',
                              'unknown'))
        
def reorg_cifar10_data(data_dir, valid_ratio):
    labels = read_csv_labels('/kaggle/input/cifar-10/trainLabels.csv')
    reorg_train_valid(data_dir, labels, valid_ratio)
    reorg_test(data_dir)

In [10]:
batch_size = 512
valid_ratio = 0.1
reorg_cifar10_data(data_dir, valid_ratio)

### Transformations
- Pre-trained networks usually apply some transformations to their inputs
- TODOs:
    - Check the transformations in the pre-trained ResNet and sync with yours! 
    - Consider adding more transformations, like HorizontalFlip, mnRandomSizeCrop

In [33]:

transform_norm = torchvision.transforms.Compose([
    torchvision.transforms.ToTensor(),
    torchvision.transforms.Normalize([0.4914, 0.4822, 0.4465],
                                     [0.2023, 0.1994, 0.2010])
])

transform_train = torchvision.transforms.Compose([
    torchvision.transforms.RandomResizedCrop(size=(32, 32),scale=(0.08, 1.0), ratio=(0.5,1.25), antialias=True),
    torchvision.transforms.ToTensor(),
    torchvision.transforms.Normalize([0.4914, 0.4822, 0.4465],
                                     [0.2023, 0.1994, 0.2010])
])

In [34]:
train_ds, train_valid_ds = [torchvision.datasets.ImageFolder(
    os.path.join(data_dir, 'train_valid_test', folder),
    transform=transform_train) for folder in ['train', 'train_valid']]

valid_ds, test_ds = [torchvision.datasets.ImageFolder(
    os.path.join(data_dir, 'train_valid_test', folder),
    transform=transform_norm) for folder in ['valid', 'test']]


train_iter, train_valid_iter = [torch.utils.data.DataLoader(
    dataset, batch_size, shuffle=True, drop_last=True)
    for dataset in (train_ds, train_valid_ds)]


valid_iter = torch.utils.data.DataLoader(valid_ds, batch_size, shuffle=False,
                                         drop_last=True)

test_iter = torch.utils.data.DataLoader(test_ds, batch_size, shuffle=False,
                                        drop_last=False)

## TODO: 
* Define the data augmentation technique 
* Fine-tune a pre-trained ResNet-18 on CIFAR-10
* Submit your submission file to Kaggle (as described in the competition rules) and get your accuracy on the test set

In [37]:
pretrained_net = torchvision.models.resnet18(pretrained=True)
pretrained_net.fc = nn.Linear(pretrained_net.fc.in_features, 10)
nn.init.xavier_normal_(pretrained_net.fc.weight)
nn.init.constant_(pretrained_net.fc.bias, 0)

device = 'cuda' if torch.cuda.is_available() else 'cpu'
pretrained_net = pretrained_net.to(device)
if device == 'cuda':
    pretrained_net = torch.nn.DataParallel(pretrained_net) # if multiple GPUs use them

config = dict(
  epochs=20,
  classes=10,
  model = pretrained_net,
  optimizer = torch.optim.Adam(pretrained_net.parameters()),
  loss = nn.CrossEntropyLoss(),
  batch_size=batch_size,
  learning_rate="default",
  dataset="CIFAR10",
  architecture="ResNet-18",
  run = "With resize crop scale=(0.08, 1.0), ratio=(0.5,1.25)",
  device = device,
)



scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(config["optimizer"], 'min', patience=3, verbose=True, min_lr=1e-5, factor=0.5)

In [25]:
#import wandb
#wandb.login()
from tqdm.notebook import trange, tqdm
run = 0

wandb: WARNING Calling wandb.login() after wandb.init() has no effect.


In [ ]:
run += 1
wandb.init(project="pretraining-resnet18", config = config, name = config["run"])

epochs = config["epochs"]
optimizer = config["optimizer"]
criterion = config["loss"]

for epoch in trange(epochs):
    
    accuracy_train = 0
    total_train = 0
    losses_train = 0
    
    pretrained_net.train()
    for X, y in train_iter:
        X = X.to(device)
        y = y.to(device)
        y_pred = pretrained_net(X)
        loss = criterion(y_pred, y)
        predicted = torch.argmax(y_pred, 1)
        accuracy_train += (y == predicted).sum().float()
        losses_train += loss.item()
        total_train += len(y)
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
    
    valid_accuracy_aug = 0
    valid_total_aug = 0
    valid_losses_aug = 0
    
    #clear 
    pretrained_net.eval()
    with torch.no_grad():
        for X, y in train_valid_iter:
            X = X.to(device)
            y = y.to(device)
            y_pred = pretrained_net(X)
            loss = criterion(y_pred, y)
            predicted = torch.argmax(y_pred, 1)
            valid_accuracy_aug += (y == predicted).sum().float()
            valid_losses_aug += loss.item()
            valid_total_aug += len(y)
    
    
    valid_acc = 0
    valid_total = 0
    valid_losses = 0
    
    pretrained_net.eval()
    with torch.no_grad():
        for X, y in valid_iter:
            X = X.to(device)
            y = y.to(device)
            y_pred = pretrained_net(X)
            loss = criterion(y_pred, y)
            predicted = torch.argmax(y_pred, 1)
            valid_acc += (y == predicted).sum().float()
            valid_losses += loss.item()
            valid_total += len(y)
            
'''
        wandb.log({
                'loss train': losses_train / len(train_iter),
                'accuracy train': accuracy_train / total_train,
             
                'valid loss aug': valid_losses_aug / len(train_valid_iter),
                'valid accuracy aug': valid_accuracy_aug / valid_total_aug,
            
                'valid loss': valid_losses / len(valid_iter),
                'valid accuracy': valid_acc / valid_total
        })
'''            


In [16]:
classes = ['airplane',
'automobile', 
'bird', 
'cat', 
'deer', 
'dog', 
'frog', 
'horse', 
'ship', 
'truck']

In [19]:
with torch.no_grad():
    pretrained_net.eval()
    test_pred = torch.LongTensor()
    for i, data in enumerate(test_iter):
        img,_ = data
        img = img.to(device)
        output = pretrained_net(img)
        output = torch.argmax(output, 1)
        output = output.cpu()
        test_pred = torch.cat((test_pred, output), dim=0)
        
label = []
for i in test_pred:
    label.append(classes[i.item()]) 
    
ids = [i for i in range(1,len(label)+1)]
out = {"id": ids, 'label': label}
out = pd.DataFrame.from_dict(out)
out.to_csv('submision.csv', index=False)